In [ ]:
%%capture
!pip install xstac

In [63]:
# List all zarr data in test-data
%load_ext autoreload
%autoreload
import json
import pystac
from pystac_client import Client
import s3fs
import sys; sys.path.append('..');
import helpers.eodc_hub_role as eodc_hub_role
import helpers.s3helpers as s3helpers
import helpers.stac as stachelpers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
credentials = eodc_hub_role.fetch_and_set_credentials()
bucket = 'nasa-eodc-data-store'
prefix = "test-data"
s3_fs = s3fs.S3FileSystem(
    key=credentials['AccessKeyId'],
    secret=credentials['SecretAccessKey'],
    token=credentials['SessionToken'], 
    anon=False
)

In [31]:
# Create a new STAC Catalog
stac_catalog = pystac.Catalog(
    id='eodc-data-store-for-zarr-viz',
    description='EODC Data Store for Zarr Viz'
)

In [32]:
suffixes = [".zarr", ".json"]
matching_paths = s3helpers.list_s3_paths(bucket, prefix, suffixes)

for path in matching_paths:
    print(path)

nasa-eodc-data-store/test-data/cmip6-kerchunk/combined_CMIP6_daily_GISS-E2-1-G_tas_kerchunk.json
nasa-eodc-data-store/test-data/cmip6-zarr/365_262_262_CMIP6_daily_GISS-E2-1-G_tas.zarr
nasa-eodc-data-store/test-data/cmip6-zarr/600_1440_1_CMIP6_daily_GISS-E2-1-G_tas.zarr
nasa-eodc-data-store/test-data/cmip6-zarr/600_1440_29_CMIP6_daily_GISS-E2-1-G_tas.zarr
nasa-eodc-data-store/test-data/fake-data/single_chunk/store_lat1024_lon2048.zarr
nasa-eodc-data-store/test-data/fake-data/single_chunk/store_lat1448_lon2896.zarr
nasa-eodc-data-store/test-data/fake-data/single_chunk/store_lat2048_lon4096.zarr
nasa-eodc-data-store/test-data/fake-data/single_chunk/store_lat2896_lon5792.zarr
nasa-eodc-data-store/test-data/fake-data/single_chunk/store_lat4096_lon8192.zarr
nasa-eodc-data-store/test-data/fake-data/single_chunk/store_lat512_lon1024.zarr
nasa-eodc-data-store/test-data/fake-data/single_chunk/store_lat724_lon1448.zarr
nasa-eodc-data-store/test-data/fake-data/with_chunks/store_lat1448_lon2896.zar

In [64]:
for path in matching_paths:
    reference = False
    if path.endswith('.json'):
        reference = True
    stac_collection = stachelpers.generate_stac(f's3://{path}', reference=reference)
    stac_collection.links = [pystac.Link(target="collection.json", rel="self")]
    stac_catalog.add_child(stac_collection)

/home/jovyan/tile-benchmarking/02-generate-stac/../titiler_xarray/titiler/xarray/reader.py:45: UserWarning: Variable(s) referenced in cell_measures not in variables: ['areacella']
  return xarray.open_dataset(src_path, **xr_open_args)
/home/jovyan/tile-benchmarking/02-generate-stac/../titiler_xarray/titiler/xarray/reader.py:45: UserWarning: Variable(s) referenced in cell_measures not in variables: ['areacella']
  return xarray.open_dataset(src_path, **xr_open_args)
/home/jovyan/tile-benchmarking/02-generate-stac/../titiler_xarray/titiler/xarray/reader.py:45: UserWarning: Variable(s) referenced in cell_measures not in variables: ['areacella']
  return xarray.open_dataset(src_path, **xr_open_args)
/home/jovyan/tile-benchmarking/02-generate-stac/../titiler_xarray/titiler/xarray/reader.py:45: UserWarning: Variable(s) referenced in cell_measures not in variables: ['areacella']
  return xarray.open_dataset(src_path, **xr_open_args)


In [65]:
with open('external-datasets.json', 'r') as f:
    external_datasets = json.loads(f.read())

for key, dataset in external_datasets.items():
    reference = False
    path = dataset['source']
    if path.endswith('.json'):
        reference = True
    stac_collection = stachelpers.generate_stac(path, idstr=key, reference=reference)
    stac_collection.links = [pystac.Link(target="collection.json", rel="self")]
    stac_catalog.add_child(stac_collection)

/home/jovyan/tile-benchmarking/02-generate-stac/../titiler_xarray/titiler/xarray/reader.py:45: UserWarning: Variable(s) referenced in cell_measures not in variables: ['areacella']
  return xarray.open_dataset(src_path, **xr_open_args)


In [66]:
stac_catalog

<Catalog id=eodc-data-store-for-zarr-viz>

In [67]:
stac_catalog.normalize_and_save(root_href="stac")

In [68]:
stac_catalog.save(pystac.CatalogType.SELF_CONTAINED)

In [69]:
STAC_API_URL = "stac/catalog.json"
catalog = Client.open(STAC_API_URL)

In [75]:
collections = list(catalog.get_collections())
for collection in sorted(collections, key=lambda x: x.id):
    print(collection.summaries.to_dict())

{'other': {'number_coord_chunks': 3, 'tas': {'chunks': [365, 262, 262], 'dtype': 'float32', 'chunk_size_mb': 95.57746887207031, 'compression': "Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)", 'total_array_size': 2406.005859375}}}
{'other': {'number_coord_chunks': 3, 'tas': {'chunks': [365, 262, 262], 'dtype': 'float32', 'chunk_size_mb': 95.57746887207031, 'compression': "Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)", 'total_array_size': 2406.005859375}}}
{'other': {'number_coord_chunks': 3, 'tas': {'chunks': [1, 600, 1440], 'dtype': 'float32', 'chunk_size_mb': 3.2958984375, 'compression': "Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)", 'total_array_size': 2406.005859375}}}
{'other': {'number_coord_chunks': 3, 'tas': {'chunks': [1, 600, 1440], 'dtype': 'float32', 'chunk_size_mb': 3.2958984375, 'compression': "Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)", 'total_array_size': 2406.005859375}}}
{'other': {'number_coord_chunks': 3, 'ta